In [1]:
# Dev env for fetching historic election data

'''
Looking for --> By district
- District stats as of 2010
    - Area (figure out units)
    - Percent of population AI
- District boundaries (simplified, separate data file)
- Control history (party and lawmaker name in district since 2014) --> DEFER UNTIL LATER
- 2020 election primary results for district --> contested/non-contested, opponents, margin 
- 2020 election general results for district --> contested/non-contested, opponents, margin
- 2020 general elex governor results --> Votes D/R/Lib

Inputs: 
- 2010 redistricting files for H and S - https://leg.mt.gov/districting/2010-districting-commission/2010-districting-maps/legislative-redistricting-plan-2013/
- 2020 governor results by house district - https://sosmt.gov/wp-content/uploads/State_Canvass_by_HD.pdf
- 2020 election results by district

Ouput 1: [ Array of house/senate districts
 - key (HD 1, HD 2, HD 10, HD 100, etc.)
 - Area (sq miles)
 - 2010 population
 - 2010 AI population
 - 2010 % of population AI
 - general2020
     - [ Gov votes ] --> { party, votes }
     - [Leg candidates, sorted by votes] --> { name, party, votes }
 - primary2020
     - Primary [candidates] { name, party, votes }

Output 2: (TODO)
- Simplified GeoJson w/ boundaries + district key in props

]
'''

'\nLooking for --> By district\n- District stats as of 2010\n    - Area (figure out units)\n    - Percent of population AI\n- District boundaries (simplified, separate data file)\n- Control history (party and lawmaker name in district since 2014) --> DEFER UNTIL LATER\n- 2020 election primary results for district --> contested/non-contested, opponents, margin \n- 2020 election general results for district --> contested/non-contested, opponents, margin\n- 2020 general elex governor results --> Votes D/R/Lib\n\nInputs: \n- 2010 redistricting files for H and S - https://leg.mt.gov/districting/2010-districting-commission/2010-districting-maps/legislative-redistricting-plan-2013/\n- 2020 governor results by house district - https://sosmt.gov/wp-content/uploads/State_Canvass_by_HD.pdf\n- 2020 election results by district\n\nOuput 1: [ Array of house/senate districts\n - key (HD 1, HD 2, HD 10, HD 100, etc.)\n - Area (sq miles)\n - 2010 population\n - 2010 AI population\n - 2010 % of populati

In [2]:
import json
from math import ceil
import pandas as pd
import geopandas as gpd

In [3]:
# Functions
def parse_sheet(xl, sheet):
    raw = xl.parse(sheet, skiprows=6)
    columns = list(raw.columns)
    race_name = columns[0]
    candidate_cols =  columns[2:]
    candidates = list(map(lambda n: n.split('\n')[0], candidate_cols))
    parties = list(map(lambda n: n.split('\n')[1], candidate_cols))
    df = raw.drop(race_name, 1)
    df = df[df['County'] == 'TOTALS'].melt(id_vars='County', value_name='votes')
    df['name'] = df['variable'].apply(lambda s: s.split('\n')[0])
    df['party'] =  df['variable'].apply(lambda s: s.split('\n')[1])
    df['party'] = df['party'].replace({
        'Democrat': 'D',
        'Republican': 'R',
        'Libertarian': 'L',
    })
    df['race'] = race_name
    df['race'] = df['race'].replace({
        'STATE SENATOR DISTRICT': 'SD',
        'STATE REPRESENTATIVE DISTRICT': 'HD',
    }, regex=True)
    return df[['race','name','party','votes']]
  

def parse_sheets(file):
    xl = pd.ExcelFile(file)
    sheets = xl.sheet_names
    df = pd.DataFrame()
    for sheet in sheets:
        dfi = parse_sheet(xl, sheet)
        df = df.append(dfi)

    return df

def hd_to_sd(key):
    # Return key for Senate District encompassing given House District
    hd_num = int(key.replace('HD ',''))
    sd_num = ceil(hd_num / 2)
    return f'SD {sd_num}'

def sd_to_hds(key):
    # Return array of two keys for House Districts within given Senate District
    sd_num = int(key.replace('SD ',''))
    return [f'HD {sd_num*2-1}', f'HD {sd_num*2}']

def collect_data_house(hd):
    district = house_geo[house_geo['key'] == hd].iloc[0]
    gov_gen = gov_20_general[gov_20_general['hd'] == hd].sort_values('votes', ascending=False)
    leg_gen = leg_20_general[leg_20_general['race'] == hd].sort_values('votes', ascending=False)
    leg_pri = leg_20_primary[leg_20_primary['race'] == hd].sort_values('votes', ascending=False)
    return {
        'key': hd,
        'area': float(district['AREA']), # Square miles
         'topology': {
            'type': 'hd',
            'related': [hd_to_sd(hd)],
        },
        'pop_2010': int(district['POPULATION']),
        'ai_pop_2010': int(district['AP_IND']),
        'last_election': '2020',
        'pri_elex': {
            'leg': leg_pri[['name','party','votes']].to_dict(orient='records')
        },
        'gen_elex': {
            'gov': gov_gen[['name','party','votes']].to_dict(orient='records'),
            'leg': leg_gen[['name','party','votes']].to_dict(orient='records')
        },
        
    }



def collect_data_senate(sd):
    district = senate_geo[senate_geo['key'] == sd].iloc[0]
    gov_gen = gov_20_general[gov_20_general['sd'] == sd]\
        .groupby(['name','party']).agg(votes=('votes','sum')).reset_index()\
        .sort_values('votes', ascending=False)

    last_election = '2020' if sd in sds_in_cycle_2020 else '2018'
    if (last_election == '2020'):
        leg_pri = leg_20_primary[leg_20_primary['race'] == sd].sort_values('votes', ascending=False)
    else:
        leg_pri = leg_18_primary[leg_18_primary['race'] == sd].sort_values('votes', ascending=False)
    
    if (last_election == '2020'):
        leg_gen = leg_20_general[leg_20_general['race'] == sd].sort_values('votes', ascending=False)
    else:
        leg_gen = leg_18_general[leg_18_general['race'] == sd].sort_values('votes', ascending=False)
    
    return {
        'key': sd,
        'topology': {
            'type': 'sd',
            'related': sd_to_hds(sd),
        },
        'area': float(district['AREA']), # Square miles
        'pop_2010': int(district['POPULATION']),
        'ai_pop_2010': int(district['AP_IND']),
        'last_election': last_election,
        'pri_elex': {
            'leg': leg_pri[['name','party','votes']].to_dict(orient='records')
        },
        'gen_elex': {
            'gov': gov_gen[['name','party','votes']].to_dict(orient='records'),
            'leg': leg_gen[['name','party','votes']].to_dict(orient='records')
        },
        
    }

In [4]:
# Raw data inputs
house_geo = gpd.read_file('raw/House_shape_adopted021213/')
house_geo['key'] = house_geo['DISTRICT'].astype(int).sort_values().apply(lambda s: f'HD {s}')

senate_geo = gpd.read_file('raw/Senate_shape_adopted021213/')
senate_geo['key'] = senate_geo['DISTRICT'].astype(int).sort_values().apply(lambda s: f'SD {s}')

gov_20_general = pd.read_csv('raw/gov-votes-by-hd-2020-gen.csv')
gov_20_general = gov_20_general.melt(id_vars='hd', var_name='name', value_name='votes')
gov_20_general['party'] = gov_20_general['name'].replace({
    'Bishop': 'L',
    'Cooney': 'D',
    'Gianforte': 'R',
})
gov_20_general['sd'] = gov_20_general['hd'].map(hd_to_sd)

leg_18_primary = parse_sheets('raw/legislative-results-2018-pri.xlsx')
leg_18_general = parse_sheets('raw/legislative-results-2018-gen.xlsx')
leg_20_primary = parse_sheets('raw/legislative-results-2020-pri.xlsx')
leg_20_general = parse_sheets('raw/legislative-results-2020-gen.xlsx')

sds_in_cycle_2020 = [f'SD {i}' for i in [2, 3, 6, 7, 10, 15, 16, 17, 18, 21, 23, 25, 26, 28, 31, 35, 36, 37, 38, 39, 40, 44, 45, 46, 47]]

In [19]:
hds = list(house_geo['key'].values)
house_data = [collect_data_house(hd) for hd in hds]

sds = list(senate_geo['key'].values)
senate_data = [collect_data_senate(sd) for sd in sds]

combined = house_data + senate_data

with open('raw/district-locales.json') as f:
    locales = json.load(f)
    
for obj in combined:
    key = obj['key']
    match = [d for d in locales if d['key'] == key][0]
    if not match: print('no match', key)
    obj['locale'] = match['mtfp_residence']
    obj['locale_description'] = match['district_scope']

with open('process/districts.json', 'w') as f:
    stringed = json.dumps(combined, indent=4)
    f.write(stringed)

In [18]:
combined

[{'key': 'HD 1',
  'area': 380.868835,
  'topology': {'type': 'hd', 'related': ['SD 1']},
  'pop_2010': 9838,
  'ai_pop_2010': 264,
  'last_election': '2020',
  'pri_elex': {'leg': [{'name': 'STEVE GUNDERSON',
     'party': 'R',
     'votes': 2121},
    {'name': 'MARVIN C SATHER', 'party': 'D', 'votes': 880}]},
  'gen_elex': {'gov': [{'name': 'Gianforte', 'party': 'R', 'votes': 3568},
    {'name': 'Cooney', 'party': 'D', 'votes': 1499},
    {'name': 'Bishop', 'party': 'L', 'votes': 210}],
   'leg': [{'name': 'STEVE GUNDERSON', 'party': 'R', 'votes': 3721},
    {'name': 'MARVIN C SATHER', 'party': 'D', 'votes': 1518}]},
  'locale': 'Libby',
  'locale_description': 'Libby, Troy'},
 {'key': 'HD 8',
  'area': 9.997384,
  'topology': {'type': 'hd', 'related': ['SD 4']},
  'pop_2010': 9989,
  'ai_pop_2010': 254,
  'last_election': '2020',
  'pri_elex': {'leg': [{'name': 'JOHN FULLER', 'party': 'R', 'votes': 2051}]},
  'gen_elex': {'gov': [{'name': 'Gianforte', 'party': 'R', 'votes': 4045},
 